# 📍 Example 4: Maximum Coverage Location Problem (MCLP) using Gurobi

This example demonstrates solving the **Maximum Coverage Location Problem** using **Gurobi** in Python. The goal is to strategically locate a limited number of facilities to maximize the total weighted demand coverage within a specified service distance.

MCLP is crucial in **facility location planning** for emergency services, retail stores, healthcare facilities, and public infrastructure placement.

---

## 🔢 Mathematical Formulation

We want to:

**Maximize:**
$$\sum_{i \in I} w_i \cdot y_i$$

**Subject to:**

**Coverage Constraints:**
$$y_i \leq \sum_{j \in S_i} x_j, \quad \forall i \in I$$

**Facility Limit:**
$$\sum_{j \in J} x_j \leq p$$

**Binary Variables:**
$$x_j \in \{0,1\}, \quad \forall j \in J$$
$$y_i \in \{0,1\}, \quad \forall i \in I$$

**Where:**
- $x_j$ = 1 if facility is located at site $j$, 0 otherwise
- $y_i$ = 1 if demand point $i$ is covered, 0 otherwise  
- $w_i$ = weight/importance of demand point $i$
- $S_i$ = set of facility sites that can cover demand point $i$
- $p$ = maximum number of facilities to locate
- $I$ = set of demand points, $J$ = set of potential facility sites

---

## 📊 Problem Instance

**Demand Points:** D1, D2, D3, D4  
**Potential Facilities:** F1, F2, F3  
**Maximum Facilities:** 2  
**Coverage Distance:** 3 units

**Distance Matrix:**

| Demand/Facility | F1 | F2 | F3 |
|-----------------|----|----|----|
| **D1**          | 2  | 4  | 5  |
| **D2**          | 3  | 2  | 6  |
| **D3**          | 4  | 3  | 2  |
| **D4**          | 5  | 6  | 3  |

**Demand Weights:**
- D1: 10 (population/importance)
- D2: 20  
- D3: 30
- D4: 40
- **Total Potential Coverage:** 100

**Coverage Sets (facilities within 3 units):**
- D1 can be covered by: F1, F2
- D2 can be covered by: F1, F2  
- D3 can be covered by: F2, F3
- D4 can be covered by: F3

---

🎯 This is a **discrete facility location problem** where we select the best 2 facilities to maximize weighted demand coverage.

In [ ]:
# MCLP Model

import gurobipy as gp
from gurobipy import GRB

# Define the data
I = ['D1', 'D2', 'D3', 'D4']
J = ['F1', 'F2', 'F3']
d = {
    ('D1', 'F1'): 2, ('D1', 'F2'): 4, ('D1', 'F3'): 5,
    ('D2', 'F1'): 3, ('D2', 'F2'): 2, ('D2', 'F3'): 6,
    ('D3', 'F1'): 4, ('D3', 'F2'): 3, ('D3', 'F3'): 2,
    ('D4', 'F1'): 5, ('D4', 'F2'): 6, ('D4', 'F3'): 3
}
coverage_distance = 3
S = {i: [j for j in J if d[i, j] <= coverage_distance] for i in I}
p = 2
w = {'D1': 10, 'D2': 20, 'D3': 30, 'D4': 40}

# Create a new model
model = gp.Model("MCLP")

# Create variables
x = model.addVars(J, vtype=GRB.BINARY, name="x")
y = model.addVars(I, vtype=GRB.BINARY, name="y")

# Set objective
model.setObjective(gp.quicksum(w[i] * y[i] for i in I), GRB.MAXIMIZE)

# Add constraints
model.addConstrs((y[i] <= gp.quicksum(x[j] for j in S[i]) for i in I), name="Coverage")
model.addConstr(gp.quicksum(x[j] for j in J) <= p, name="FacilityLimit")

# Optimize model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for j in J:
        if x[j].x > 0.5:
            print(f"Facility located at {j}")
    for i in I:
        if y[i].x > 0.5:
            print(f"Demand point {i} is covered")
else:
    print("No optimal solution found")


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 7 columns and 13 nonzeros
Model fingerprint: 0x95e97aca
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 5 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 2: 100 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+02, best bound 1.000000000000e+02, gap 0.0000%
Optimal solution found:
Facility located at F1
Faci